In [1]:
import numpy as np

def soil_temperature_profile(T_surface, A_0, w, D, z, t, t0=0):
    """
    Computes the soil temperature at a given depth and time.
    
    Parameters:
    - T_surface: Average surface temperature (°C)
    - A_0: Surface temperature amplitude (°C)
    - w: Angular frequency (rad/s)
    - D: Damping depth (m)
    - z: Depth in soil (m)
    - t: Time (s)
    - t0: Phase shift (s), default is 0
    
    Returns:
    - Temperature at depth z and time t (°C)
    """
    return T_surface + A_0 * np.exp(-z / D) * np.cos(w * t - z / D - t0)

def damping_depth(thermal_diffusivity, angular_frequency):
    """
    Calculates the damping depth.
    
    Parameters:
    - thermal_diffusivity: Thermal diffusivity of the soil (m²/s)
    - angular_frequency: Angular frequency of temperature fluctuations (rad/s)
    
    Returns:
    - Damping depth (m)
    """
    return np.sqrt(2 * thermal_diffusivity / angular_frequency)


In [2]:
def volumetric_heat_capacity(theta, rho_b, c_mineral, rho_mineral, c_water=4.18, c_organic=1.92, rho_organic=0):
    """
    Calculates volumetric heat capacity of the soil.
    
    Parameters:
    - theta: Water content (m³/m³)
    - rho_b: Bulk density (g/cm³)
    - c_mineral: Specific heat of minerals (J/g/°C)
    - rho_mineral: Density of minerals (g/cm³)
    - c_water: Specific heat of water (J/g/°C), default is 4.18
    - c_organic: Specific heat of organic matter (J/g/°C), default is 1.92
    - rho_organic: Density of organic matter (g/cm³), default is 0
    
    Returns:
    - Volumetric heat capacity (MJ/m³/°C)
    """
    phi_mineral = rho_b / rho_mineral
    return phi_mineral * c_mineral + theta * c_water + (1 - phi_mineral - theta) * rho_organic * c_organic

def thermal_diffusivity(thermal_conductivity, volumetric_heat_capacity):
    """
    Computes the thermal diffusivity of the soil.
    
    Parameters:
    - thermal_conductivity: Thermal conductivity (W/m/°C)
    - volumetric_heat_capacity: Volumetric heat capacity (MJ/m³/°C)
    
    Returns:
    - Thermal diffusivity (m²/s)
    """
    return thermal_conductivity / volumetric_heat_capacity


In [3]:
def heat_flux_density(A_0, k, w, D, t, t0=0):
    """
    Calculates the heat flux density at the soil surface.
    
    Parameters:
    - A_0: Surface temperature amplitude (°C)
    - k: Thermal conductivity (W/m/°C)
    - w: Angular frequency (rad/s)
    - D: Damping depth (m)
    - t: Time (s)
    - t0: Phase shift (s), default is 0
    
    Returns:
    - Heat flux density (W/m²)
    """
    return A_0 * k / D * np.sin(w * t - t0 + np.pi / 4)


In [4]:
def interface_temperature(T1, T2, p1, p2):
    """
    Computes the temperature at the interface of two materials.
    
    Parameters:
    - T1: Temperature of material 1 (°C)
    - T2: Temperature of material 2 (°C)
    - p1: Thermal admittance of material 1
    - p2: Thermal admittance of material 2
    
    Returns:
    - Interface temperature (°C)
    """
    return (T1 * p1 + T2 * p2) / (p1 + p2)


In [5]:
import matplotlib.pyplot as plt

def plot_temperature_depth(T_surface, A_0, w, D, z_range, t):
    """
    Plots temperature vs. depth for given time.
    
    Parameters:
    - T_surface: Surface temperature (°C)
    - A_0: Surface temperature amplitude (°C)
    - w: Angular frequency (rad/s)
    - D: Damping depth (m)
    - z_range: Array of depths (m)
    - t: Time (s)
    """
    temperatures = [soil_temperature_profile(T_surface, A_0, w, D, z, t) for z in z_range]
    plt.plot(temperatures, z_range)
    plt.gca().invert_yaxis()
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Depth (m)')
    plt.title('Temperature vs Depth')
    plt.show()


In [6]:
# Example 8.1: Volumetric Heat Capacity Calculation for Loam Soil
# Parameters
water_content = 0.2  # m³/m³
bulk_density = 1.3  # g/cm³
c_mineral = 0.87  # J/g/°C
rho_mineral = 2.65  # g/cm³
c_water = 4.18  # J/g/°C

# Volumetric heat capacity calculation
phi_mineral = bulk_density / rho_mineral  # Mineral fraction
volumetric_heat_capacity = phi_mineral * c_mineral + water_content * c_water  # MJ/m³/°C

# Example 8.2: Thermal Conductivity Calculation
# Parameters
temp = 20  # °C
pa = 101000  # Pa (Atmospheric pressure)
ka_air = 0.025  # W/m/°C (Thermal conductivity of air)
kw = 0.6  # W/m/°C (Thermal conductivity of water)
q = 4  # Dimensionless flow factor
theta = water_content  # Volume fraction of water
phi_mineral = bulk_density / rho_mineral
phi_gas = 1 - theta - phi_mineral  # Volume fraction of gas

# Latent heat transport calculations
latent_heat = 44100  # J/mol
slope_saturation = 145  # Pa/°C
D_v = 2.42e-5  # m²/s (Vapor diffusivity)
vapor_pressure = 2340  # Pa

# Vapor phase conductivity
k_vapor = ka_air + (latent_heat * D_v * vapor_pressure * slope_saturation) / (pa ** 2)

# Fluid conductivity and weighting factors
fluid_conductivity = phi_gas * k_vapor + theta * kw
g_gas = 0.1  # Shape factor for gas
g_water = 1 - 2 * g_gas
k_thermal = fluid_conductivity * (g_gas * phi_gas + g_water * theta)

volumetric_heat_capacity, k_thermal


(1.2627924528301886, 0.02439241417317501)

In [7]:
def thermal_conductivity(phi_water, phi_gas, phi_mineral, k_water, k_air, k_mineral, g_gas=0.1):
    """
    Calculates the thermal conductivity of soil based on DeVries' method (Equation 8.13).
    
    Parameters:
    - phi_water: Volume fraction of water (m³/m³)
    - phi_gas: Volume fraction of gas (m³/m³)
    - phi_mineral: Volume fraction of minerals (m³/m³)
    - k_water: Thermal conductivity of water (W/m/°C)
    - k_air: Thermal conductivity of air (W/m/°C)
    - k_mineral: Thermal conductivity of mineral material (W/m/°C)
    - g_gas: Shape factor for gas (default is 0.1)
    
    Returns:
    - Thermal conductivity of the soil (W/m/°C)
    """
    g_water = 1 - 2 * g_gas  # Shape factor for water
    k_soil = (
        phi_gas * g_gas * k_air +
        phi_water * g_water * k_water +
        phi_mineral * (1 - g_gas - g_water) * k_mineral
    )
    return k_soil



In [8]:
# Example parameters
phi_water = 0.2  # Volume fraction of water
phi_gas = 0.31  # Volume fraction of gas
phi_mineral = 0.49  # Volume fraction of minerals
k_water = 0.6  # W/m/°C
k_air = 0.025  # W/m/°C
k_mineral = 2.5  # W/m/°C

# Calculate thermal conductivity
thermal_k = thermal_conductivity(phi_water, phi_gas, phi_mineral, k_water, k_air, k_mineral)
print(f"Thermal Conductivity: {thermal_k:.4f} W/m/°C")


Thermal Conductivity: 0.2193 W/m/°C


In [9]:
from scipy.special import erf
import numpy as np

def temperature_profile(T_surface, T_initial, diffusivity, depth, time):
    """
    Computes the temperature profile in a semi-infinite medium (Equation 8.21).

    Parameters:
    - T_surface: Surface temperature after sudden change (°C)
    - T_initial: Initial temperature of the medium (°C)
    - diffusivity: Thermal diffusivity of the medium (m²/s)
    - depth: Depth in the medium (m)
    - time: Time elapsed since the surface temperature change (s)

    Returns:
    - Temperature at the given depth and time (°C)
    """
    return T_initial + (T_surface - T_initial) * erf(depth / (2 * np.sqrt(diffusivity * time)))


In [10]:
# Example parameters
T_surface = 37  # Surface temperature (°C)
T_initial = 20  # Initial temperature (°C)
diffusivity = 0.14e-6  # Thermal diffusivity (m²/s)
depth = 0.05  # Depth in the medium (m)
time = 3600  # Time elapsed (s)

# Calculate temperature profile
temperature = temperature_profile(T_surface, T_initial, diffusivity, depth, time)
print(f"Temperature at depth {depth} m and time {time} s: {temperature:.2f} °C")


Temperature at depth 0.05 m and time 3600 s: 35.04 °C


In [11]:
def heat_flux_surface(T_surface, T_initial, thermal_admittance, time):
    """
    Computes the heat flux at the surface of a semi-infinite medium (Equation 8.22).
    
    Parameters:
    - T_surface: Surface temperature after sudden change (°C)
    - T_initial: Initial temperature of the medium (°C)
    - thermal_admittance: Thermal admittance of the medium (W/m²/K/s^0.5)
    - time: Time elapsed since the surface temperature change (s)
    
    Returns:
    - Heat flux at the surface (W/m²)
    """
    return thermal_admittance * (T_surface - T_initial) / np.sqrt(np.pi * time)


In [12]:
# Example parameters
T_surface = 37  # Surface temperature (°C)
T_initial = 20  # Initial temperature (°C)
thermal_admittance = 100  # Thermal admittance (W/m²/K/s^0.5)
time = 3600  # Time elapsed (s)

# Calculate surface heat flux
heat_flux = heat_flux_surface(T_surface, T_initial, thermal_admittance, time)
print(f"Heat flux at the surface after {time} seconds: {heat_flux:.2f} W/m²")


Heat flux at the surface after 3600 seconds: 15.99 W/m²


In [13]:
def average_soil_conductance(thermal_admittance, contact_time):
    """
    Computes the average thermal conductance of the soil (Equation 8.23).
    
    Parameters:
    - thermal_admittance: Thermal admittance of the soil (W/m²/K/s^0.5)
    - contact_time: Duration of contact with the soil (s)
    
    Returns:
    - Average thermal conductance (W/m²/K)
    """
    return 2 * thermal_admittance / np.sqrt(np.pi * contact_time)


In [14]:
# Example parameters
thermal_admittance = 100  # Thermal admittance (W/m²/K/s^0.5)
contact_time = 3600  # Contact time (s)

# Calculate average soil conductance
soil_conductance = average_soil_conductance(thermal_admittance, contact_time)
print(f"Average soil conductance over {contact_time} seconds: {soil_conductance:.2f} W/m²/K")


Average soil conductance over 3600 seconds: 1.88 W/m²/K
